In [4]:
%matplotlib ipympl
from QDP import qdp
import numpy as np
import matplotlib.pyplot as plt
import os.path
from scipy.stats import skew, skewtest
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]

In [64]:
exp_name = '2018_07_19_14_25_41_csrb-collisions-42-high-density-pt4'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[35],[3]]
rb_cuts = [[32],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            fmt=['bo','rx'][r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_07_19/2018_07_19_14_25_41_csrb-collisions-42-high-density-pt4/results.hdf5
ivars: []
Could not find conditional retention
ivar:	0.0	retention:[0.76031746 0.79554937 2.5       ] +- [0.01264909 0.01264909 0.01264909]
Only one iteration


FigureCanvasNbAgg()

In [65]:
drop_itr = []

dual_data = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
indv_data = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
dual_err = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
indv_err = np.zeros((2, len(q.experiments[0]['iterations'].keys()) - len(drop_itr)))
dual_ret_data = []
j = 0

for i in q.experiments[0]['iterations'].keys():
    if i in drop_itr:
        print('Skipping iteration: {}'.format(i))
        j += 1
        continue

    cs_cuts = [[35],[3]]
    rb_cuts = [[32],[3]]
    q.set_thresholds(cs_cuts, roi=0)
    q.set_thresholds(rb_cuts, roi=1)
    results = q.generate_thresholds(itr=i, loss=False)
    
    itr_cuts = {
        0: q.get_thresholds()[0],
        1: q.get_thresholds()[1]
    }
    print(itr_cuts)
    q.apply_thresholds(exclude_rois=exclude)
    qd = q.experiments[0]['iterations'][i]['quantized_data']
    dual_load = np.logical_and(qd[:,0,0], qd[:,0,1])
    print(np.sum(dual_load))
    cs_only = np.logical_and(qd[:,0,0], np.logical_not(qd[:,0,1]))
    rb_only = np.logical_and(np.logical_not(qd[:,0,0]), qd[:,0,1])
    dual_ret =  np.mean(qd[dual_load,1], axis=0)[:2]
    cs_ret = np.mean(qd[cs_only,1], axis=0)[:2]
    rb_ret = np.mean(qd[rb_only,1], axis=0)[:2]
    
    dual_ret_data.append(qd[dual_load,1])

    dual_data[:, i-j] = dual_ret
    indv_data[0, i-j] = cs_ret[0]
    indv_data[1, i-j] = rb_ret[1]

    n_dual = sum(dual_load)
    n_cs = sum(cs_only)
    n_rb = sum(rb_only)

    def binomial_err(n, p):
        return np.sqrt(p*(1-p)/n)

    dual_err[:, i-j] = binomial_err(n_dual, np.mean(qd[dual_load,1], axis=0))[:2]
    cs_err = binomial_err(n_cs, np.mean(qd[cs_only,1], axis=0))[0]
    rb_err = binomial_err(n_rb, np.mean(qd[rb_only,1], axis=0))[1]
    indv_err[0, i-j] = cs_err
    indv_err[1, i-j] = rb_err

--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[3.01145486e-18 0.00000000e+00 0.00000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[3.01145486e-18 0.00000000e+00 0.00000000e+00]`
--------------------
{0: [[31], [2]], 1: [[31], [4]]}
Could not find conditional retention
151


In [66]:
dual_data

array([[0.54304636],
       [0.54304636]])

In [67]:
indv_data

array([[0.8562753 ],
       [0.84938704]])

In [68]:
fig, ax = plt.subplots(ncols=2)
for r in range(2):
    ax[r].errorbar(
        np.arange(dual_data.shape[1]), dual_data[r],
        yerr=dual_err[r],
        fmt=['bo','rx'][r],
        label=['cs(rb)', 'rb(cs)'][r]
    )
    ax[r].errorbar(
        0.2+np.arange(dual_data.shape[1]), indv_data[r],
        yerr=indv_err[r],
        fmt=['cs','yd'][r],
        label=['cs(!rb)', 'rb(!cs)'][r]
    )
    # plot weighted means
    indv_wmean = np.dot(indv_data[r], np.power(indv_err[r], -2))/np.sum(np.power(indv_err[r], -2))
    indv_stderr = np.sqrt(np.sum(np.power(indv_err[r], 2)))/dual_data.shape[1]
    dual_wmean = np.dot(dual_data[r], np.power(dual_err[r], -2))/np.sum(np.power(dual_err[r], -2))
    dual_stderr = np.sqrt(np.sum(np.power(dual_err[r], 2)))/dual_data.shape[1]
    print ("{}_data.append([{},{},{},{}])".format(['cs','rb'][r],indv_wmean, indv_stderr, dual_wmean, dual_stderr))
    ax[r].axhline(indv_wmean, color='cy'[r], alpha=0.5, lw=3)
    ax[r].axhline(dual_wmean, color='br'[r], alpha=0.5, lw=3)
    # plot uncertainty of the mean
    ax[r].fill_between([0, dual_data.shape[1]], indv_wmean-indv_stderr, indv_wmean+indv_stderr, facecolor='cy'[r], alpha=0.2)
    ax[r].fill_between([0, dual_data.shape[1]], dual_wmean-dual_stderr, dual_wmean+dual_stderr, facecolor='br'[r], alpha=0.2)
    
    ax[r].set_xlabel(xlab)
    ax[r].legend(bbox_to_anchor=(0.95, 0.15), loc=1, borderaxespad=0.)
    ax[r].set_ylim(0., 1)
ax[0].set_ylabel('Retention')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'dual_retention_stability.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

cs_data.append([0.856275303644,0.0157837042504,0.543046357616,0.04053834838])
rb_data.append([0.84938704028,0.014968074378,0.543046357616,0.04053834838])


In [69]:
ps = np.zeros((2,2))
n = 0
for itr in dual_ret_data:
    n += len(itr)
    ps[1,1] += np.sum(np.logical_and(itr[:,0], itr[:,1]))  # both atoms retained
    ps[0,0] += np.sum(np.logical_and(np.logical_not(itr[:,0]), np.logical_not(itr[:,1])))  # both atoms lost
    ps[0,1] += np.sum(np.logical_and(np.logical_not(itr[:,0]), itr[:,1]))  # cs only lost
    ps[1,0] += np.sum(np.logical_and(itr[:,0], np.logical_not(itr[:,1])))  # cs only lost
ps = ps.astype('float')/n

In [70]:
print('corr_ret_data.append(['+','.join(['[{},{}]'.format(ps[i][0],ps[i][1]) for i in range(2)])+'])')
print('corr_ret_err.append(['+','.join(['[{},{}]'.format(binomial_err(n, ps)[i][0],binomial_err(n, ps)[i][1]) for i in range(2)])+'])')

corr_ret_data.append([[0.35761589404,0.0993377483444],[0.0993377483444,0.443708609272]])
corr_ret_err.append([[0.0390047345037,0.0243416303385],[0.0243416303385,0.0404307337798]])


In [71]:
binomial_err(n,1./(2*n))

0.004675066912975902

In [72]:
#ps11 = []

In [73]:
bins = 40
trials = 25
fig, ax = plt.subplots()


for i in q.experiments[0]['iterations'].keys():
    dual_data = np.zeros((2, 1))
    indv_data = np.zeros((2, 1))
    dual_err = np.zeros((2, 1))
    indv_err = np.zeros((2, 1))
    dual_ret_data = []

    cs_cuts = [[35],[3]]
    rb_cuts = [[32],[3]]
    q.set_thresholds(cs_cuts, roi=0)
    q.set_thresholds(rb_cuts, roi=1)
    results = q.generate_thresholds(itr=i, loss=False)

    itr_cuts = {
        0: q.get_thresholds()[0],
        1: q.get_thresholds()[1]
    }
    print(itr_cuts)
    q.apply_thresholds(exclude_rois=exclude)
    qd = q.experiments[0]['iterations'][i]['quantized_data']
    dual_load = np.logical_and(qd[:,0,0], qd[:,0,1])
    dual_data =  qd[dual_load,1,:2]

    for k in range(1):
        ps = np.zeros((2,2,bins))

        for nn in xrange(bins):
            itr = dual_data[np.random.randint(len(dual_data), size=trials)]
            ps[1,1,nn] += np.log(np.sum(np.logical_and(itr[:,0], itr[:,1])).astype('float')/trials)  # both atoms retained
            ps[0,0,nn] += np.sum(np.logical_and(np.logical_not(itr[:,0]), np.logical_not(itr[:,1]))).astype('float')/trials  # both atoms lost
            ps[0,1,nn] += np.sum(np.logical_and(np.logical_not(itr[:,0]), itr[:,1])).astype('float')/trials  # cs only lost
            ps[1,0,nn] += np.sum(np.logical_and(itr[:,0], np.logical_not(itr[:,1]))).astype('float')/trials  # cs only lost

        ax.hist(ps[1,1], alpha=0.3)
        ps11.append(ps[1,1])

FigureCanvasNbAgg()

--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[3.01145486e-18 0.00000000e+00 0.00000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[3.01145486e-18 0.00000000e+00 0.00000000e+00]`
--------------------
{0: [[31], [2]], 1: [[31], [4]]}
Could not find conditional retention


In [109]:
ts = np.array([8.5,30,60,100,300,600,1200])/1000.
fig, ax = plt.subplots(nrows=len(ps11), figsize=(3,8), sharex=True)
ks = []
for n, d in enumerate(ps11):
    k = -np.array(d)/np.sqrt(ts[n])
    ax[n].hist(k, alpha=0.8, normed=True)
#     ax[n].set_xlim(-1.5,0.1)
    print(np.mean(k))
    ks.append(np.mean(k))
    ax[n].text(20, 0.05, r"$t={}$ s".format(ts[n]))
ax[-1].set_xlabel(r"$-ln(p(t))/\sqrt{t}$")
ax[3].set_ylabel(r"PDF")
plt.tight_layout()

fig2, ax2 = plt.subplots()
ax2.plot(ts, ks)

FigureCanvasNbAgg()

1.2197461351814725
1.4137923203638503
0.9446377232673526
0.9730423630078635
0.7112670008967481
1.0037196304474003
0.7353642566809804


FigureCanvasNbAgg()

In [23]:
fig, ax = plt.subplots(nrows=len(ps11), figsize=(3,8), sharex=True)
for n, d in enumerate(ps11):
    ax[n].hist(d, alpha=0.8)
    ax[n].set_xlim(0,1)

FigureCanvasNbAgg()